In [ ]:
import os
os._exit(0)

In [1]:
!pip install streamlit pyngrok pandas numpy matplotlib seaborn shap plotly scikit-learn pyngrok --quiet
!pip install bcrypt --quiet
!pip install streamlit pyngrok pandas numpy matplotlib seaborn shap plotly scikit-learn pyngrok --quiet
!pip install bcrypt --quiet
!pip install shap --quiet
!pip install pyngrok --quiet
!pip install streamlit pandas numpy matplotlib seaborn plotly statsmodels scikit-learn shap --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.8 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import json
import csv
from datetime import datetime
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import shap
import base64
import sqlite3
import datetime
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from pyngrok import ngrok
import bcrypt
import io
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

def init_db():
    conn = sqlite3.connect("users.db")

    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS users (
            username TEXT PRIMARY KEY,
            password BLOB NOT NULL,
            is_admin INTEGER DEFAULT 0 -- Added is_admin column
        )
    ''')
    c.execute('''
        CREATE TABLE IF NOT EXISTS activity_logs (
            username TEXT,
            timestamp TEXT,
            activity TEXT
        )
    ''')
    conn.commit()
    conn.close()
init_db()
def add_user(username, password, is_admin=0):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    hashed_pass = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
    try:
        c.execute("INSERT INTO users (username, password, is_admin) VALUES (?, ?, ?)", (username, hashed_pass, is_admin))
        conn.commit()
        conn.close()
        return True
    except sqlite3.IntegrityError:
        conn.close()
        return False
def check_login(username, password):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT password, is_admin FROM users WHERE username = ?", (username,))
    result = c.fetchone()
    conn.close()
    if result and bcrypt.checkpw(password.encode(), result[0]):
        return True, bool(result[1])
    return False, False
def log_user_activity(username, activity):
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.execute("INSERT INTO activity_logs (username, timestamp, activity) VALUES (?, ?, ?)", (username, timestamp, activity))
    conn.commit()
    conn.close()
def get_activity_logs():
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT username, timestamp, activity FROM activity_logs ORDER BY timestamp DESC")
    logs = c.fetchall()
    conn.close()
    return logs
def get_all_users():
    conn = sqlite3.connect("users.db")
    c = conn.cursor()
    c.execute("SELECT username, is_admin FROM users")
    users = c.fetchall()
    conn.close()
    return users
def log_activity(username, activity):
    pass
def log_session(username, session_type):
    pass
def get_dashboard_content(username):
    return {}
def update_dashboard_content(username, content):
    pass
def get_user_content(username):
    conn = sqlite3.connect("user_content.db")
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS user_content
                 (username TEXT PRIMARY KEY, content TEXT)''')
    conn.commit()
    c.execute("SELECT content FROM user_content WHERE username = ?", (username,))
    result = c.fetchone()
    conn.close()
    if result:
        return json.loads(result[0])
    return {}
def update_user_content(username, content):
    conn = sqlite3.connect("user_content.db")
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS user_content
                 (username TEXT PRIMARY KEY, content TEXT)''')
    conn.commit()
    c.execute("INSERT OR REPLACE INTO user_content (username, content) VALUES (?, ?)", (username, json.dumps(content)))
    conn.commit()
    conn.close()
def get_visible_pages():
    conn = sqlite3.connect("app_settings.db")
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS page_visibility
                 (page_name TEXT PRIMARY KEY, is_visible INTEGER)''')
    conn.commit()
    default_pages = ["Home", "About", "SACCO Interface", "Budgeting", "Literature", "M-Pesa Monitor"]
    for page in default_pages:
        c.execute("INSERT OR IGNORE INTO page_visibility (page_name, is_visible) VALUES (?, ?)", (page, 1))
    conn.commit()

    c.execute("SELECT page_name, is_visible FROM page_visibility")
    visibility_data = c.fetchall()
    conn.close()
    return {page: bool(visible) for page, visible in visibility_data}
def set_page_visibility(page_name, is_visible):
    conn = sqlite3.connect("app_settings.db")
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS page_visibility
                 (page_name TEXT PRIMARY KEY, is_visible INTEGER)''')
    conn.commit()
    c.execute("INSERT OR REPLACE INTO page_visibility (page_name, is_visible) VALUES (?, ?)", (page_name, 1 if is_visible else 0))
    conn.commit()
    conn.close()
st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Open+Sans&family=Playfair+Display:ital,wght@1,700&family=Poppins:wght@500&family=Roboto&display=swap');

    html, body, .stApp {
        background: linear-gradient(to bottom right, #2e7d32, #ffffff, #4e342e);  /* green → white → brown */
        font-family: 'Roboto', sans-serif;
        color: #000000;  /* black text for contrast */
        font-size: 18px;
    }

    .main {
        background: rgba(255, 255, 255, 0.9);  /* light background for content area */
        border-radius: 15px;
        padding: 25px;
        box-shadow: 0 4px 12px rgba(0,0,0,0.1);
    }

    h1, h2, h3, h4, h5, h6 {
        font-family: 'Playfair Display', serif;
        color: #0d47a1 !important; /* vivid blue for headers */
        font-weight: bold;
        font-style: italic;
        font-size: 28px;
    }

    .stButton>button {
        font-family: 'Poppins', sans-serif;
        background-color: #2e7d32; /* green */
        color: #ffffff;
        font-weight: bold;
        font-size: 18px;
        border-radius: 8px;
        padding: 8px 16px;
        transition: background-color 0.3s ease;
        border: none;
    }
    .stButton>button:hover {
        background-color: #1b5e20; /* darker green on hover */
    }

    .stTextInput>div>div>input {
        font-family: 'Open Sans', sans-serif;
        background-color: #ffffff;
        color: #000000;
        border-radius: 5px;
        font-size: 16px;
        padding: 8px;
        border: 1px solid #a1887f;
    }

    .stAlert {
        background-color: #fce4ec;
        color: #4e342e;
        font-size: 16px;
        border-left: 6px solid #2e7d32;
        padding: 10px;
    }

    .stMarkdown p, .stMarkdown a {
        font-family: 'Roboto', sans-serif;
        color: #000000 !important; /* black text for all content */
        font-size: 18px;
    }

    .stSidebar {
        background-color: #4e342e !important; /* brown */
        color: #ffffff;
    }

    .stSidebar .css-1d391kg {
        color: #ffffff !important;
    }
    </style>
""", unsafe_allow_html=True)
if 'logged_in' not in st.session_state:
    st.session_state.logged_in = False
if 'username' not in st.session_state:
    st.session_state.username = ''
if 'is_admin' not in st.session_state:
    st.session_state.is_admin = False
if 'nav_selection' not in st.session_state:
    st.session_state.nav_selection = 'Login'
page_visibility = get_visible_pages()
if st.session_state.logged_in:
    menu = []
    if page_visibility.get("Home", True): menu.append("Home")
    if page_visibility.get("About", True): menu.append("About")
    if page_visibility.get("SACCO Interface", True): menu.append("SACCO Interface")
    if page_visibility.get("Budgeting", True): menu.append("Budgeting")
    if page_visibility.get("Literature", True): menu.append("Literature")
    if page_visibility.get("Mobile money Monitor", True): menu.append("Mobile money Monitor")
    menu.append("Logout")

    if st.session_state.is_admin:
        menu.append("Admin Dashboard")
else:
    menu = ["Login", "Sign Up"]

nav_default = st.session_state.get("nav_selection")
if nav_default not in menu:
    nav_default = menu[0]

choice = st.sidebar.selectbox("Navigation", menu, index=menu.index(nav_default))
st.session_state.nav_selection = choice

if choice == "Login":
    st.title("Login to ElewaPesa")
    login_type = st.radio("Login as:", ("User", "Admin"))

    username = st.text_input("Username")
    password = st.text_input("Password", type='password')
    if st.button("Login"):
        if login_type == "User":
            success, is_admin = check_login(username, password)
            if success:
                if is_admin and login_type == "User":
                     st.error("Invalid user login details")
                else:
                    st.session_state.logged_in = True
                    st.session_state.username = username
                    st.session_state.is_admin = is_admin
                    log_user_activity(username, "Logged in as User")
                    st.success(f"Welcome back, {username}! Redirecting to Home...")
                    st.session_state.nav_selection = "Home"
                    st.rerun()
            else:
                st.error("Invalid user login details")
        elif login_type == "Admin":
            conn = sqlite3.connect("users.db")
            c = conn.cursor()
            c.execute("SELECT password FROM users WHERE username = ? AND is_admin = 1", (username,))
            result = c.fetchone()
            conn.close()

            if result and bcrypt.checkpw(password.encode(), result[0]):
                st.session_state.logged_in = True
                st.session_state.username = username
                st.session_state.is_admin = True
                log_user_activity(username, "Logged in as Admin")
                st.success("Welcome Admin!")
                log_activity(st.session_state.username, 'Viewed Dashboard')
                st.session_state.nav_selection = "Admin Dashboard"
                st.rerun()
            else:
                st.error("Invalid admin login details")

elif choice == "Sign Up":
    st.title("Sign Up")
    new_user = st.text_input("Create Username")
    new_pass = st.text_input("Create Password", type='password')

    signup_type = st.radio("Sign up as:", ("User", "Admin"))

    if st.button("Sign Up"):
        if new_user and new_pass:
            is_admin_value = 1 if signup_type == "Admin" else 0
            if add_user(new_user, new_pass, is_admin=is_admin_value):
                st.success(f"{signup_type} account created! You can now log in.")
                log_user_activity(new_user, f"Signed up as {signup_type}")
            else:
                st.error("Username already exists. Please choose another.")
        else:
            st.error("Please fill in all fields.")

elif choice == "Logout":
    if st.session_state.logged_in:
      log_session(st.session_state.username, 'logout')
      log_user_activity(st.session_state.username, "Logged out") # Log logout
      st.session_state.logged_in = False
      st.session_state.username = ''
      st.session_state.is_admin = False
      st.success("You have been logged out.")
      st.session_state.nav_selection = "Login"
      st.rerun()
elif choice == "Admin Dashboard":
    if st.session_state.is_admin:
        log_activity(st.session_state.username, 'Viewed Admin Dashboard')
        st.title("Administrator Dashboard")

        st.markdown("### Registered Users")
        users = get_all_users()
        if users:
            user_df = pd.DataFrame(users, columns=["Username", "Is Admin"])
            user_df['Is Admin'] = user_df['Is Admin'].apply(lambda x: 'Yes' if x else 'No')
            st.dataframe(user_df)
        else:
            st.info("No users registered yet.")

        st.markdown("### User Activity Logs")
        logs = get_activity_logs()
        if logs:
            log_df = pd.DataFrame(logs, columns=["Username", "Timestamp", "Activity"])
            st.dataframe(log_df)
        else:
            st.info("No user activity logs found.")

        st.markdown("### Manage User Content (Placeholder)")
        st.warning("This is a placeholder. Implementing full content management requires a more robust database structure and UI.")
        all_users = [user[0] for user in get_all_users()]
        selected_user_for_content = st.selectbox("Select a user to manage content for:", ["-- Select User --"] + all_users)

        if selected_user_for_content != "-- Select User --":
            user_current_content = get_user_content(selected_user_for_content)
            st.write(f"Current content for {selected_user_for_content}:")
            st.json(user_current_content)

            new_content_json = st.text_area("Enter new content (JSON format):", json.dumps(user_current_content, indent=2))
            if st.button(f"Update content for {selected_user_for_content}"):
                try:
                    updated_content = json.loads(new_content_json)
                    update_user_content(selected_user_for_content, updated_content)
                    st.success(f"Content updated for {selected_user_for_content}.")
                    log_user_activity(st.session_state.username, f"Updated content for {selected_user_for_content}")
                    st.rerun()
                except json.JSONDecodeError:
                    st.error("Invalid JSON format. Please check your input.")

            if st.button(f"Clear all content for {selected_user_for_content}"):
                 update_user_content(selected_user_for_content, {})
                 st.success(f"All content cleared for {selected_user_for_content}.")
                 log_user_activity(st.session_state.username, f"Cleared content for {selected_user_for_content}")
                 st.rerun()


        st.markdown("### Manage Page Visibility")
        current_visibility = get_visible_pages()
        st.write("Control which pages are visible to non-admin users.")
        for page, is_visible in current_visibility.items():
            new_visibility = st.checkbox(f"Show '{page}' page", value=is_visible, key=f"visibility_{page}")
            if new_visibility != is_visible:
                set_page_visibility(page, new_visibility)
                st.info(f"Visibility of '{page}' page updated. Changes will be reflected on the next page load.")
                log_user_activity(st.session_state.username, f"Set visibility of '{page}' to {new_visibility}")
                st.rerun()

        st.markdown("### Add New Admin User")
        new_admin_user = st.text_input("New Admin Username")
        new_admin_pass = st.text_input("New Admin Password", type='password')
        if st.button("Add Admin"):
            if new_admin_user and new_admin_pass:
                if add_user(new_admin_user, new_admin_pass, is_admin=1):
                    st.success(f"Admin user '{new_admin_user}' added.")
                    log_user_activity(st.session_state.username, f"Added new admin user: {new_admin_user}")
                else:
                    st.error("Username already exists.")
            else:
                st.error("Please fill in all fields.")

    else:
        st.warning("You do not have permission to access this page.")
        st.session_state.nav_selection = "Home"
        st.rerun()

elif choice == "Home":
    if page_visibility.get("Home", True) or st.session_state.is_admin:
        if st.session_state.logged_in:
          log_activity(st.session_state.username, 'Viewed Dashboard')
          st.title("ElewaPesa Dashboard")
          log_activity(st.session_state.username, 'Viewed Dashboard')
          st.markdown("Welcome to your personalized financial dashboard, where you can manage budgets, analyze SACCO data, and track M-Pesa transactions.")
          col1, col2 = st.columns(2)
          with col1:
                if page_visibility.get("SACCO Interface", True) or st.session_state.is_admin:
                    if st.button("SACCO Interface"):
                        st.session_state.nav_selection = "SACCO Interface"
                        st.rerun()
                    st.caption("Upload and analyze SACCO data using statistical and machine learning tools.")
                if page_visibility.get("Literature", True) or st.session_state.is_admin:
                    if st.button("Literature"):
                        st.session_state.nav_selection = "Literature"
                        st.rerun()
                    st.caption("Explore curated financial literacy resources and guides.")
                if page_visibility.get("About", True) or st.session_state.is_admin:
                    if st.button("About"):
                        st.session_state.nav_selection = "About"
                        st.rerun()
                    st.caption("Learn more about ElewaPesa and its mission.")
          with col2:
            if page_visibility.get("Budgeting", True) or st.session_state.is_admin:
                if st.button("Budgeting"):
                    st.session_state.nav_selection = "Budgeting"
                    st.rerun()
                st.caption("Use the budgeting tool to allocate income across categories.")

            if page_visibility.get("Mobile money Monitor", True) or st.session_state.is_admin:
                if st.button("Mobile money Monitor"):
                    st.session_state.nav_selection = "Mobile money Monitor"
                    st.rerun()
                st.caption("Monitor Mobile money transactions and gain insights into your spending.")
    else:
         st.warning("This page is currently hidden by the administrator.")
         st.session_state.nav_selection = "Login"
         st.rerun()
elif choice == "About":
    if page_visibility.get("About", True) or st.session_state.is_admin:
        if st.session_state.logged_in:
            st.title("About ElewaPesa")
            st.markdown("""
                **ElewaPesa** is a financial literacy platform designed to empower users with tools for:
                - Budgeting and financial education
                - SACCO data analysis and forecasting
                - Real-time M-Pesa monitoring and reporting
                - Secure login and personalized dashboards
            """)

            with st.expander("Vision and Mission"):
                st.markdown("""
                **Vision:** To become the leading financial consultancy and budgeting solutions provider for SACCOs and Kenyans.

                **Mission:** To empower financial growth through data-driven tools, personalized insights, and accessible financial education.
                """)

            st.markdown("### Services Offered by Elewa Pesa")
            services = {
                "Financial Consultancy for SACCOs": [
                    "Strategic financial planning & advisory",
                    "Customized budget optimization",
                    "Debt & loan portfolio evaluation",
                    "Compliance and reporting support"
                ],
                "Personal Budgeting Tools": [
                    "Income, expense & savings tracking",
                    "Smart budget templates for Kenyan households",
                    "Financial goal setting and tracking",
                    "Alerts for overspending and savings progress"
                ],
                "Data-Driven Financial Analysis": [
                    "Upload and analyze SACCO or personal data",
                    "Financial ratios, trends, and heatmaps",
                    "Forecasting using ARIMA",
                    "Clustering and behavioral insights"
                ],
                "Financial Literacy Resources": [
                    "Interactive guides & explainers",
                    "Personalized literacy assessments",
                    "Educational content for SACCO members",
                    "Budgeting workshops & community modules"
                ],
                "Real-Time MOBILE MONEY Integration": [
                    "Live transaction tracking",
                    "Automatic income/expense classification",
                    "Mobile money dashboard visualization",
                    "Integration with budget analytics"
                ],
                "Secure User Account & Data Management": [
                    "Encrypted logins and data privacy",
                    "User-specific dashboards & reports",
                    "Downloadable CSV summaries",
                    "Persistent financial history"
                ]
            }
            for section, features in services.items():
                with st.expander(section):
                    for feature in features:
                        st.markdown(f"- {feature}")
            st.markdown("#### Why Financial Literacy Matters")
            st.info("""
            Financial literacy enables individuals to make informed financial decisions, avoid debt traps, and plan for their future.
            In Kenya, SACCOs have become pivotal in improving access to financial services, and understanding their structure
            and offerings is vital for inclusive growth.
            """)

            st.markdown("Contact Us")
            st.markdown("""
            - **Email:** [support@elewapesa.com](mailto:support@elewapesa.com)
            - **Phone:** +254 712 345 678
            - **WhatsApp:** [Click to chat](https://wa.me/254712345678)
            - **Twitter:** [@ElewaPesa](https://twitter.com/ElewaPesa)
            - **Facebook:** [Elewa Pesa](https://facebook.com/ElewaPesa)
            - **Instagram:** [@elewa.pesa](https://instagram.com/elewa.pesa)
            """)
    else:
        st.warning("This page is currently hidden by the administrator.")
        st.session_state.nav_selection = "Home"
        st.rerun()

elif choice == "SACCO Interface":
    if page_visibility.get("SACCO Interface", True) or st.session_state.is_admin:
        if st.session_state.logged_in:
            import matplotlib.pyplot as plt
            import seaborn as sns
            import plotly.express as px
            from scipy import stats
            from statsmodels.tsa.seasonal import seasonal_decompose
            from statsmodels.tsa.arima.model import ARIMA
            from sklearn.cluster import KMeans
            from sklearn.preprocessing import StandardScaler
            import shap
            import pandas as pd

            st.title("SACCO Data Upload & Analysis")

            uploaded_file = st.file_uploader("Upload your SACCO data (CSV format)", type="csv")

            if uploaded_file is not None:
                log_user_activity(st.session_state.username, "Uploaded SACCO data")  # Log upload
                df = pd.read_csv(uploaded_file)

                if 'User_ID' in df.columns:
                    df = df.drop(columns=['User_ID'])

                st.dataframe(df.head())

                required_cols = ['Date', 'Income', 'Expenses', 'Savings']
                if not all(col in df.columns for col in required_cols):
                    st.info(f"Financial analysis requires columns: {', '.join(required_cols)}.")
                    st.stop()

                df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
                df = df.dropna(subset=['Date'])
                df = df.sort_values('Date').set_index('Date')

                st.sidebar.header("Filters")
                if 'Region' in df.columns:
                    regions = df['Region'].dropna().unique()
                    selected_regions = st.sidebar.multiselect("Select Region(s)", options=regions, default=list(regions))
                else:
                    selected_regions = []

                date_min = df.index.min()
                date_max = df.index.max()
                selected_date_range = st.sidebar.date_input("Select Date Range", value=(date_min, date_max), min_value=date_min, max_value=date_max)

                if selected_regions and 'Region' in df.columns:
                    df = df[df['Region'].isin(selected_regions)]
                df = df.loc[selected_date_range[0]:selected_date_range[1]]

                if df.empty:
                    st.warning("No data available after applying filters.")
                    st.stop()

                st.markdown("### Summary Statistics")
                st.write(df.describe())

                st.markdown("### Financial Ratios")
                df['Savings_Rate'] = df['Savings'] / df['Income'].replace(0, pd.NA)
                df['Expense_Ratio'] = df['Expenses'] / df['Income'].replace(0, pd.NA)
                st.write(df[['Savings_Rate', 'Expense_Ratio']].describe())

                df_monthly = df.resample('M').sum()

                st.markdown("### Monthly Aggregated Income, Expenses and Savings")
                fig_monthly, ax_monthly = plt.subplots(figsize=(10, 6))
                df_monthly[['Income', 'Expenses', 'Savings']].plot(ax=ax_monthly)
                ax_monthly.set_title("Monthly Aggregated Income, Expenses and Savings")
                st.pyplot(fig_monthly)

                st.markdown("### Distribution of Income")
                fig_income, ax_income = plt.subplots()
                sns.histplot(df['Income'], kde=True, ax=ax_income, bins=30)
                st.pyplot(fig_income)

                st.markdown("### Distribution of Savings")
                fig_savings, ax_savings = plt.subplots()
                sns.histplot(df['Savings'], kde=True, ax=ax_savings, bins=30)
                st.pyplot(fig_savings)

                st.markdown("### Box Plots of Income, Expenses and Savings")
                fig_box, ax_box = plt.subplots()
                sns.boxplot(data=df[['Income', 'Expenses', 'Savings']], ax=ax_box)
                st.pyplot(fig_box)

                st.markdown("### Scatter Plot: Income vs Expenses colored by Savings")
                fig_scatter, ax_scatter = plt.subplots()
                sc = ax_scatter.scatter(df['Income'], df['Expenses'], c=df['Savings'], cmap='viridis', alpha=0.7)
                plt.colorbar(sc, ax=ax_scatter, label='Savings')
                ax_scatter.set_xlabel('Income')
                ax_scatter.set_ylabel('Expenses')
                st.pyplot(fig_scatter)

                st.markdown("### Savings Rate Over Time")
                fig_srate, ax_srate = plt.subplots()
                df['Savings_Rate'].plot(ax=ax_srate)
                ax_srate.set_title("Savings Rate Over Time")
                st.pyplot(fig_srate)

                if 'Region' in df.columns and selected_regions:
                    st.markdown("### Average Income by Region")
                    avg_income = df.groupby('Region')['Income'].mean().loc[selected_regions]
                    fig_reg_income = px.bar(avg_income, title="Average Income by Region")
                    st.plotly_chart(fig_reg_income)

                    st.markdown("### Average Savings by Region")
                    avg_savings = df.groupby('Region')['Savings'].mean().loc[selected_regions]
                    fig_reg_savings = px.bar(avg_savings, title="Average Savings by Region")
                    st.plotly_chart(fig_reg_savings)

                st.markdown("### Cumulative Income vs Cumulative Expenses Over Time")
                cum_df = df[['Income', 'Expenses']].cumsum()
                fig_cum, ax_cum = plt.subplots()
                cum_df.plot(ax=ax_cum)
                st.pyplot(fig_cum)

                st.markdown("### Hypothesis Test: Expenses Before vs After Budgeting Start Date")
                budget_start = st.date_input("Select budgeting start date", value=date_min)
                before = df[df.index < pd.to_datetime(budget_start)]['Expenses']
                after = df[df.index >= pd.to_datetime(budget_start)]['Expenses']
                if len(before) > 1 and len(after) > 1:
                    t_stat, p_val = stats.ttest_ind(before, after, equal_var=False)
                    st.write(f"t-statistic = {t_stat:.3f}, p-value = {p_val:.3f}")
                    if p_val < 0.05:
                        st.success("Significant difference detected in expenses after budgeting started.")
                    else:
                        st.info("No significant difference detected in expenses after budgeting started.")
                else:
                    st.info("Not enough data to perform hypothesis test.")

                st.markdown("### Income Seasonality Decomposition")
                try:
                    decomposition = seasonal_decompose(df['Income'], model='additive', period=12)
                    fig_decomp = decomposition.plot()
                    st.pyplot(fig_decomp)
                except Exception as e:
                    st.warning(f"Seasonality decomposition failed: {e}")

                st.markdown("### Income Forecast (next 6 months)")
                try:
                    model = ARIMA(df['Income'], order=(1, 1, 1))
                    model_fit = model.fit()
                    forecast = model_fit.forecast(steps=6)
                    forecast_index = pd.date_range(df.index[-1], periods=7, freq='M')[1:]
                    forecast_series = pd.Series(forecast, index=forecast_index)
                    fig_forecast, ax_forecast = plt.subplots(figsize=(10, 5))
                    df['Income'].plot(ax=ax_forecast, label='Historical Income')
                    forecast_series.plot(ax=ax_forecast, style='--', label='Forecast')
                    ax_forecast.legend()
                    st.pyplot(fig_forecast)
                    st.write(forecast_series)
                except Exception as e:
                    st.warning(f"Forecasting failed: {e}")

                st.markdown("### Clustering Analysis (KMeans)")
                try:
                    features = df[['Income', 'Expenses', 'Savings']].dropna()
                    scaler = StandardScaler()
                    scaled_features = scaler.fit_transform(features)
                    kmeans = KMeans(n_clusters=3, random_state=42)
                    clusters = kmeans.fit_predict(scaled_features)
                    features = features.assign(Cluster=clusters)

                    fig_cluster, ax_cluster = plt.subplots()
                    scatter = ax_cluster.scatter(features['Income'], features['Expenses'], c=features['Cluster'], cmap='Set1', alpha=0.6)
                    ax_cluster.set_xlabel('Income')
                    ax_cluster.set_ylabel('Expenses')
                    ax_cluster.set_title('KMeans Clusters of Income and Expenses')
                    legend1 = ax_cluster.legend(*scatter.legend_elements(), title="Clusters")
                    ax_cluster.add_artist(legend1)
                    st.pyplot(fig_cluster)
                except Exception as e:
                    st.warning(f"Clustering failed: {e}")

                st.markdown("### SHAP Explainability for Savings Prediction")
                try:
                    import sklearn.linear_model as lm
                    features_shap = df[['Income', 'Expenses']].dropna()
                    target_shap = df.loc[features_shap.index, 'Savings']
                    model_shap = lm.LinearRegression()
                    model_shap.fit(features_shap, target_shap)
                    explainer = shap.Explainer(model_shap, features_shap)
                    shap_values = explainer(features_shap)
                    st.pyplot(shap.summary_plot(shap_values, features_shap, show=False))
                except Exception as e:
                    st.warning(f"SHAP analysis failed: {e}")

                csv = df.reset_index().to_csv(index=False).encode()
                st.download_button(label="Download Processed SACCO Data as CSV", data=csv, file_name='processed_sacco_data.csv', mime='text/csv')

elif choice == "Budgeting":
    if st.session_state.logged_in:
        st.title("Budgeting Tool")
        income = st.number_input("Enter your total monthly income:", min_value=0)

        if income > 0:
            st.markdown("Break down your spending below:")
            rent = st.slider("Rent/Housing", 0, income, int(income*0.3))
            food = st.slider("Food", 0, income, int(income*0.2))
            transport = st.slider("Transport", 0, income, int(income*0.1))
            other = st.slider("Other Expenses", 0, income, int(income*0.1))
            total_expense = rent + food + transport + other
            remaining = income - total_expense
            st.success(f"Remaining Balance: KES {remaining}")
            fig = px.pie(names=["Rent", "Food", "Transport", "Other", "Remaining"], values=[rent, food, transport, other, remaining], title="Budget Distribution")
            st.plotly_chart(fig)
        else:
            st.info("Please enter your monthly income to access the budgeting tools.")
elif choice == "Literature":
    if st.session_state.logged_in:
        st.title("Financial Literacy Resources")
        st.image("https://www.investopedia.com/thmb/BFmdJbKhStQQ2pby0Tw0Gp-3HiM=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/GettyImages-1354286504-71e832b51e8d47a4baccb5e0bcfa0cb4.jpg", use_container_width=True)

        st.markdown("""
            ### Recommended Reads
            - [Budgeting 101 - Investopedia](https://www.investopedia.com/terms/b/budget.asp)
            - [Understanding SACCOs in Kenya](https://www.profinch.com/sacco/)
            - [Saving and Investing Tips - Capital Markets Authority Kenya](https://www.cma.or.ke/)
            - [Money Academy – Financial Literacy Curriculum](https://www.fitmoney.org/)
            - [Financial Education for African Youth](https://www.aflatoun.org/)
        """)
        st.markdown("#### Why Financial Literacy Matters")
        st.info("""
        Financial literacy enables individuals to make informed financial decisions, avoid debt traps, and plan for their future.
        In Kenya, SACCOs have become pivotal in improving access to financial services, and understanding their structure
        and offerings is vital for inclusive growth.
        """)
elif choice == "Mobile money Monitor":
     if st.session_state.logged_in:
         st.title("Real-time Mobile money Monitoring")
         st.markdown("Integration in progress... Live API connectivity will display here once credentials are set.")

Writing app.py


In [3]:
!pkill -f ngrok
!ps -ef | grep ngrok
!pip install pyngrok --quiet


root        3051    2258  0 14:21 ?        00:00:00 /bin/bash -c ps -ef | grep ngrok
root        3053    3051  0 14:21 ?        00:00:00 grep ngrok


In [4]:
import threading
import os
from pyngrok import ngrok
def run_streamlit():
    os.system("streamlit run app.py")
os.environ["BROWSER_GATHER_USAGE_STATS"] = "false"
threading.Thread(target=run_streamlit).start()
ngrok.set_auth_token("2wgwHmFlV02P3dBq1fRuuA0tW58_uB9Rz61BT6Le22PJuYzp")
public_url = ngrok.connect(8501)
print("Streamlit app is running at:", public_url)



Streamlit app is running at: NgrokTunnel: "https://5e9dcc24f8ef.ngrok-free.app" -> "http://localhost:8501"
